<a href="https://colab.research.google.com/github/ananyanivarthi/aud2text/blob/main/whisperai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-n0fj0v7p
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-n0fj0v7p
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802825 sha256=cc449c223a56ac1ed2e5ce140e396632e47a1233754512156a0799ba06dcff38
  Stored in directory: /tmp/pip-ephem-wheel-cache-6gufw884/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,

In [ ]:
pip install torchaudio


In [ ]:
import os
import subprocess
import torchaudio
from multiprocessing import Pool
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor

# Load Whisper model and processor
model_name = "openai/whisper-medium"  # Adjust the model type as needed
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

# Function to preprocess audio
def preprocess_audio(input_path, output_path):
    command = f"ffmpeg -i \"{input_path}\" -ar 16000 -ac 1 \"{output_path}\""
    subprocess.run(command, shell=True)

# Function to transcribe audio and save output
def transcribe_and_save(input_path, output_path):
    # Load audio using torchaudio
    audio_tensor, sample_rate = torchaudio.load(input_path)

    # Ensure audio is in the correct format (mono, 16000 Hz)
    if audio_tensor.size(0) > 1:  # If stereo, take only one channel
        audio_tensor = audio_tensor.mean(dim=0, keepdim=True)
    if sample_rate != 16000:  # Resample if necessary
        audio_tensor = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(audio_tensor)

    # Convert audio tensor to NumPy array and squeeze it
    audio_input = audio_tensor.squeeze().numpy()

    # Preprocess audio input
    audio_input_processed = processor(audio=audio_input, return_tensors="pt", sampling_rate=16000)

    # Perform transcription
    with torch.no_grad():
        transcription = model.generate(**audio_input_processed)

    # Decode the transcription
    transcription_text = processor.batch_decode(transcription, skip_special_tokens=True)[0]

    # Print transcription for debugging
    print(f"Transcribing: {input_path}")
    print("Transcription:", transcription_text)

    # Save transcription output
    with open(output_path, 'w') as file:
        file.write(transcription_text)

# Function to handle a single audio file
def process_audio_file(file_info):
    input_path, output_transcription_path, preprocessed_path = file_info
    preprocess_audio(input_path, preprocessed_path)
    transcribe_and_save(preprocessed_path, output_transcription_path)

# Paths and file handling
audio_files = ["/content/03-01-01-01-01-01-01.wav",
               "/content/WhatsApp Audio 2024-07-25 at 14.23.14_f8572176.waptt.opus",
               "/content/mixkit-dog-barking-twice-1.wav"]
output_dir = "/content/transcriptions"

# Create the output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Prepare file info for processing
file_info_list = []
for audio_file in audio_files:
    filename = os.path.basename(audio_file)
    preprocessed_path = os.path.join(output_dir, f"preprocessed_{filename}")
    output_transcription_path = os.path.join(output_dir, f"transcription_{filename}.txt")
    file_info_list.append((audio_file, output_transcription_path, preprocessed_path))

# Use multiprocessing to process audio files in parallel
with Pool() as pool:
    pool.map(process_audio_file, file_info_list)

# Save the model (if needed) to reuse later
torch.save(model.state_dict(), "/content/drive/MyDrive/whisper_model.pth")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. 

Transcribing: /content/transcriptions/preprocessed_03-01-01-01-01-01-01.wav
Transcription:  Kids are talking by the door.
Transcribing: /content/transcriptions/preprocessed_WhatsApp Audio 2024-07-25 at 14.23.14_f8572176.waptt.opus
Transcription:  Hello, my name is Ananya and I am a very good girl.
Transcribing: /content/transcriptions/preprocessed_mixkit-dog-barking-twice-1.wav
Transcription:  Woof! Woof!
